### Kaggle getting started - Titanic 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### EDA

In [10]:
train = pd.read_csv('train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

**Overall survival rate**

In [18]:
int(train['Survived'].value_counts()[1]/train['Survived'].value_counts()[0]*100)

62

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Try to see if the cabin letter-code bears some predictive information

In [12]:
train['C-letter'] = train[train['Cabin'].notnull()]['Cabin'].map(lambda x: str(x)[0])

In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN


In [14]:
train.groupby('C-letter').mean()['Survived']

C-letter
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
Name: Survived, dtype: float64

The letter appears useful

In [19]:
train[train['Cabin'].isnull()]['Survived'].value_counts()

0    481
1    206
Name: Survived, dtype: int64

people will NaN cabin info were less likely to survive, perhaps there were people sleeping in a shared space?

In [20]:
train[train['Age'].isnull()]['Survived'].value_counts()

0    125
1     52
Name: Survived, dtype: int64

people with no age were less likely to survive - perhaps for people who didn't survive it was more difficult to gather the data later?

In [46]:
train[train['Age']>train['Age'].mean()]['Survived'].value_counts()

0    196
1    134
Name: Survived, dtype: int64

In [51]:
train['Age'].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

There are some strange ages (fractions, then ages < 1)

In [32]:
train[train['Age']==1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
164,165,0,3,"Panula, Master. Eino Viljami",male,1.0,4,1,3101295,39.6875,NaN,S,NaN
172,173,1,3,"Johnson, Miss. Eleanor Ileen",female,1.0,1,1,347742,11.1333,NaN,S,NaN
183,184,1,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0000,F4,S,F
381,382,1,3,"Nakid, Miss. Maria (""Mary"")",female,1.0,0,2,2653,15.7417,NaN,C,NaN
386,387,0,3,"Goodwin, Master. Sidney Leonard",male,1.0,5,2,CA 2144,46.9000,NaN,S,NaN
788,789,1,3,"Dean, Master. Bertram Vere",male,1.0,1,2,C.A. 2315,20.5750,NaN,S,NaN
827,828,1,2,"Mallet, Master. Andre",male,1.0,0,2,S.C./PARIS 2079,37.0042,NaN,C,NaN


In [29]:
train[train['Cabin']=='F4']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
183,184,1,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0,F4,S,F
618,619,1,2,"Becker, Miss. Marion Louise",female,4.0,2,1,230136,39.0,F4,S,F


In [28]:
train[train['Ticket']=='230136']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
183,184,1,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0,F4,S,F
618,619,1,2,"Becker, Miss. Marion Louise",female,4.0,2,1,230136,39.0,F4,S,F


In [27]:
train[train['Name'].str.contains('Becker')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
183,184,1,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0,F4,S,F
618,619,1,2,"Becker, Miss. Marion Louise",female,4.0,2,1,230136,39.0,F4,S,F


This is somewhat strange, did these two children travel alone?
I am trying to figure out whether I can guess if the fractioned ages are children, or mistakes

In [33]:
train[train['Age']==2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,NaN
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q,NaN
119,120,0,3,"Andersson, Miss. Ellis Anna Maria",female,2.0,4,2,347082,31.2750,NaN,S,NaN
205,206,0,3,"Strom, Miss. Telma Matilda",female,2.0,0,1,347054,10.4625,G6,S,G
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.5500,C22 C26,S,C
340,341,1,2,"Navratil, Master. Edmond Roger",male,2.0,1,1,230080,26.0000,F2,S,F
479,480,1,3,"Hirvonen, Miss. Hildur E",female,2.0,0,1,3101298,12.2875,NaN,S,NaN
530,531,1,2,"Quick, Miss. Phyllis May",female,2.0,1,1,26360,26.0000,NaN,S,NaN
642,643,0,3,"Skoog, Miss. Margit Elizabeth",female,2.0,3,2,347088,27.9000,NaN,S,NaN
824,825,0,3,"Panula, Master. Urho Abraham",male,2.0,4,1,3101295,39.6875,NaN,S,NaN


In [34]:
train[train['Cabin']=='G6']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,G
205,206,0,3,"Strom, Miss. Telma Matilda",female,2.0,0,1,347054,10.4625,G6,S,G
251,252,0,3,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",female,29.0,1,1,347054,10.4625,G6,S,G
394,395,1,3,"Sandstrom, Mrs. Hjalmar (Agnes Charlotta Bengt...",female,24.0,0,2,PP 9549,16.7000,G6,S,G


In [35]:
train[train['Cabin']=='F2']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
148,149,0,2,"Navratil, Mr. Michel (""Louis M Hoffman"")",male,36.5,0,2,230080,26.0,F2,S,F
193,194,1,2,"Navratil, Master. Michel M",male,3.0,1,1,230080,26.0,F2,S,F
340,341,1,2,"Navratil, Master. Edmond Roger",male,2.0,1,1,230080,26.0,F2,S,F


In [40]:
train[train['Age']%1!=0].dropna(subset=['Age'])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
57,58,0,3,"Novel, Mr. Mansouer",male,28.50,0,0,2697,7.2292,NaN,C,NaN
78,79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,248738,29.0000,NaN,S,NaN
111,112,0,3,"Zabour, Miss. Hileni",female,14.50,1,0,2665,14.4542,NaN,C,NaN
116,117,0,3,"Connors, Mr. Patrick",male,70.50,0,0,370369,7.7500,NaN,Q,NaN
122,123,0,2,"Nasser, Mr. Nicholas",male,32.50,1,0,237736,30.0708,NaN,C,NaN
123,124,1,2,"Webber, Miss. Susan",female,32.50,0,0,27267,13.0000,E101,S,E
148,149,0,2,"Navratil, Mr. Michel (""Louis M Hoffman"")",male,36.50,0,2,230080,26.0000,F2,S,F
152,153,0,3,"Meo, Mr. Alfonzo",male,55.50,0,0,A.5. 11206,8.0500,NaN,S,NaN
153,154,0,3,"van Billiard, Mr. Austin Blyler",male,40.50,0,2,A/5. 851,14.5000,NaN,S,NaN
203,204,0,3,"Youseff, Mr. Gerious",male,45.50,0,0,2628,7.2250,NaN,C,NaN


It does appear that the ages that are less than 1 are children, based on Patch, which is mostly non=zero for those. I will proceed assuming the fractions are real ages, give as parts of a year

In [41]:
train[train['Name'].str.contains('Allison')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.55,C22 C26,S,C
305,306,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.55,C22 C26,S,C
498,499,0,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.55,C22 C26,S,C


In [43]:
train[train['Age']==0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter


In [44]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN


In [47]:
train[train['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,B
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,B


In [49]:
train[train['Ticket'].map(lambda x: x[:2])=='11'].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,C-letter
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.10,C123,S,C
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.55,C103,S,C
23,24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.50,A6,S,A
35,36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.00,NaN,S,NaN
41,42,0,2,"Turpin, Mrs. William John Robert (Dorothy Ann ...",female,27.0,1,0,11668,21.00,NaN,S,NaN


The ticket number that starts with 113 seems to be indicative of embraked in S

In [51]:
train[train['Ticket'].map(lambda x: x[:3])=='113']['Embarked'].value_counts()

S    41
C     4
Name: Embarked, dtype: int64

### Plan

I will drop ID, Name, ticket and Cabin<br>
I will add Cabin's first letter and one-hot-encode that, including a label for NaN<br>
I will one-hot-encode Embarked<br>
I will add a binary label for ageNaN?<br>
I will try NaN ages with 0, or the mean<br>
I will encode Sex<br>
I will fill missing Embarked with 'S' (for these two cases)<br>
I will try Pclass as ordinal or one-hot-encoded<br>

In [120]:
data = train.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,C-letter
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,NaN


In [121]:
data['Sex'].unique()

array(['male', 'female'], dtype=object)

In [122]:
data['Sex'] = data['Sex'].map({'male':0,'female':1})

In [123]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,C-letter
0,0,3,0,22.0,1,0,7.2500,S,NaN
1,1,1,1,38.0,1,0,71.2833,C,C
2,1,3,1,26.0,0,0,7.9250,S,NaN
3,1,1,1,35.0,1,0,53.1000,S,C
4,0,3,0,35.0,0,0,8.0500,S,NaN


In [124]:
data['AgeNull'] = data['Age'].isnull().astype(int)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,C-letter,AgeNull
0,0,3,0,22.0,1,0,7.2500,S,NaN,0
1,1,1,1,38.0,1,0,71.2833,C,C,0
2,1,3,1,26.0,0,0,7.9250,S,NaN,0
3,1,1,1,35.0,1,0,53.1000,S,C,0
4,0,3,0,35.0,0,0,8.0500,S,NaN,0


In [125]:
idx = data[data['Embarked'].isnull()].index

In [126]:
data.loc[idx]['Embarked'] = 'S'

In [127]:
data = pd.concat((data,pd.get_dummies(data['Embarked'])),axis=1).drop('Embarked',axis=1)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C-letter,AgeNull,C,Q,S
0,0,3,0,22.0,1,0,7.2500,NaN,0,0,0,1
1,1,1,1,38.0,1,0,71.2833,C,0,1,0,0
2,1,3,1,26.0,0,0,7.9250,NaN,0,0,0,1
3,1,1,1,35.0,1,0,53.1000,C,0,0,0,1
4,0,3,0,35.0,0,0,8.0500,NaN,0,0,0,1


Make sure there is no overlap between the names from Embarked and the Cabin letters

In [128]:
data['C-letter'].unique()

array([nan, 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [129]:
letters = pd.get_dummies(data['C-letter'])
letters.head(1)

,A,B,C,D,E,F,G,T
0,0,0,0,0,0,0,0,0


In [130]:
letters.columns

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'], dtype='object')

In [131]:
letters.columns = ['C-A', 'C-B', 'C-C', 'C-D', 'C-E', 'C-F', 'C-G', 'C-T']
letters.head(1)

,C-A,C-B,C-C,C-D,C-E,C-F,C-G,C-T
0,0,0,0,0,0,0,0,0


In [132]:
data = pd.concat((data,letters),axis=1).drop('C-letter',axis=1)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,AgeNull,C,Q,S,C-A,C-B,C-C,C-D,C-E,C-F,C-G,C-T
0,0,3,0,22.0,1,0,7.2500,0,0,0,1,0,0,0,0,0,0,0,0
1,1,1,1,38.0,1,0,71.2833,0,1,0,0,0,0,1,0,0,0,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,0,1,0,0,0,0,0,0,0,0
3,1,1,1,35.0,1,0,53.1000,0,0,0,1,0,0,1,0,0,0,0,0
4,0,3,0,35.0,0,0,8.0500,0,0,0,1,0,0,0,0,0,0,0,0


In [133]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 19 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
AgeNull     891 non-null int64
C           891 non-null uint8
Q           891 non-null uint8
S           891 non-null uint8
C-A         891 non-null uint8
C-B         891 non-null uint8
C-C         891 non-null uint8
C-D         891 non-null uint8
C-E         891 non-null uint8
C-F         891 non-null uint8
C-G         891 non-null uint8
C-T         891 non-null uint8
dtypes: float64(2), int64(6), uint8(11)
memory usage: 65.3 KB


In [134]:
data['Age'] = data['Age'].fillna(0)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,AgeNull,C,Q,S,C-A,C-B,C-C,C-D,C-E,C-F,C-G,C-T
0,0,3,0,22.0,1,0,7.2500,0,0,0,1,0,0,0,0,0,0,0,0
1,1,1,1,38.0,1,0,71.2833,0,1,0,0,0,0,1,0,0,0,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,0,1,0,0,0,0,0,0,0,0
3,1,1,1,35.0,1,0,53.1000,0,0,0,1,0,0,1,0,0,0,0,0
4,0,3,0,35.0,0,0,8.0500,0,0,0,1,0,0,0,0,0,0,0,0


In [136]:
data.isnull().sum().sum()

0

In [139]:
X = data.drop('Survived',axis=1).values
y = data['Survived'].values

In [147]:
np.set_printoptions(suppress=True)
np.round(X.mean(axis=0),2)

array([ 2.31,  0.35, 23.8 ,  0.52,  0.38, 32.2 ,  0.2 ,  0.19,  0.09,
        0.72,  0.02,  0.05,  0.07,  0.04,  0.04,  0.01,  0.  ,  0.  ])

Age and Fare need to be scaled (Pclass is probably small enough to be ok

In [148]:
from sklearn.preprocessing import StandardScaler

In [149]:
scaler = StandardScaler()

In [153]:
X[:,[2,5]]

array([[22.    ,  7.25  ],
       [38.    , 71.2833],
       [26.    ,  7.925 ],
       ...,
       [ 0.    , 23.45  ],
       [26.    , 30.    ],
       [32.    ,  7.75  ]])

In [154]:
scaler.fit(X[:,[2,5]])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [155]:
X_orig = X.copy()
X[:,[2,5]] = scaler.transform(X[:,[2,5]])
np.round(X.mean(axis=0),2)

array([2.31, 0.35, 0.  , 0.52, 0.38, 0.  , 0.2 , 0.19, 0.09, 0.72, 0.02,
       0.05, 0.07, 0.04, 0.04, 0.01, 0.  , 0.  ])

# Unlabeled data

In [212]:
test = pd.read_csv('test.csv')

In [213]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


One fare missing, I'll use the mean<br>
the rest, same as training set

In [214]:
test['C-letter'] = test[test['Cabin'].notnull()]['Cabin'].map(lambda x: str(x)[0])

In [215]:
tset = test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [216]:
tset['Sex'] = tset['Sex'].map({'male':0,'female':1})

In [217]:
tset['AgeNull'] = tset['Age'].isnull().astype(int)

In [218]:
tset = pd.concat((tset,pd.get_dummies(tset['Embarked'])),axis=1).drop('Embarked',axis=1)

In [219]:
tset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,C-letter,AgeNull,C,Q,S
0,3,0,34.5,0,0,7.8292,NaN,0,0,1,0
1,3,1,47.0,1,0,7.0000,NaN,0,0,0,1
2,2,0,62.0,0,0,9.6875,NaN,0,0,1,0
3,3,0,27.0,0,0,8.6625,NaN,0,0,0,1
4,3,1,22.0,1,1,12.2875,NaN,0,0,0,1


In [220]:
tset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
Pclass      418 non-null int64
Sex         418 non-null int64
Age         332 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Fare        417 non-null float64
C-letter    91 non-null object
AgeNull     418 non-null int64
C           418 non-null uint8
Q           418 non-null uint8
S           418 non-null uint8
dtypes: float64(2), int64(5), object(1), uint8(3)
memory usage: 27.4+ KB


In [223]:
idx = tset[tset['Fare'].isnull()].index

In [230]:
letters = pd.get_dummies(tset['C-letter'])
letters.columns

Index(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='object')

In [231]:
letters.columns = ['C-A', 'C-B', 'C-C', 'C-D', 'C-E', 'C-F', 'C-G']
tset = pd.concat((tset,letters),axis=1).drop('C-letter',axis=1)
tset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,AgeNull,C,Q,S,C-A,C-B,C-C,C-D,C-E,C-F,C-G
0,3,0,34.5,0,0,7.8292,0,0,1,0,0,0,0,0,0,0,0
1,3,1,47.0,1,0,7.0000,0,0,0,1,0,0,0,0,0,0,0
2,2,0,62.0,0,0,9.6875,0,0,1,0,0,0,0,0,0,0,0
3,3,0,27.0,0,0,8.6625,0,0,0,1,0,0,0,0,0,0,0
4,3,1,22.0,1,1,12.2875,0,0,0,1,0,0,0,0,0,0,0


tset is missing cabin T

In [263]:
tset['C-T'] = 0

In [232]:
tset['Age'] = tset['Age'].fillna(0)

In [235]:
tset['Fare'] = tset['Fare'].fillna(tset['Fare'].mean())

In [238]:
tset.isnull().sum().sum()

0

In [271]:
tX = tset.values

In [273]:
tX_orig = tX.copy()
tX[:,[2,5]] = scaler.transform(tX[:,[2,5]])
np.round(tX.mean(axis=0),2)

array([2.27, 0.36, 0.01, 0.45, 0.39, 0.07, 0.21, 0.24, 0.11, 0.65, 0.02,
       0.04, 0.08, 0.03, 0.02, 0.02, 0.  , 0.  ])

# Fitting

In [246]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import cross_val_score

Neural network will not be attempted for this dataset

In [314]:
estimators = {
    'Logistic':LogisticRegression(solver='liblinear'),
    'D-Tree':DecisionTreeClassifier(),
    'Forest':RandomForestClassifier(n_estimators=50),
    'N-Bayes':GaussianNB(),
    'LinearSVM':SVC(kernel='linear'),
    'rbf SVM':SVC(gamma='scale'), # default rbf
    'KNearestN':KNeighborsClassifier(),
    'XGBoost':GradientBoostingClassifier(),
    'AdaBoost':AdaBoostClassifier()
}

In [280]:
for estimator in estimators:
    print(estimator,':',end='')
    estimators[estimator].fit(X,y)
    print('\t',round(cross_val_score(estimators[estimator],X,y,cv=5).mean(),4))
    #print('\t',round(estimators[estimator].score(X,y),4))

Logistic :	 0.7947
D-Tree :	 0.7621
Forest :	 0.797
N-Bayes :	 0.5217
LinearSVM :	 0.7845
rbf SVM :	 0.807
KNearestN :	 0.7846
XGBoost :	 0.8227
AdaBoost :	 0.8092


**Try to convert age 0 to mean**

In [288]:
X_orig[X_orig[:,2]!=0][:,2].mean()

29.69911764705882

In [311]:
X2 = X_orig.copy()
X2[X_orig[:,2]==0][:,2] = X_orig[X_orig[:,2]!=0][:,2].mean()

In [312]:
scaler2 = StandardScaler()
scaler2.fit(X2[:,[2,5]])
X2[:,[2,5]] = scaler2.transform(X2[:,[2,5]])

In [315]:
for estimator in estimators:
    print(estimator,':',end='')
    estimators[estimator].fit(X2,y)
    print('\t',round(cross_val_score(estimators[estimator],X2,y,cv=5).mean(),4))
    #print('\t',round(estimators[estimator].score(X,y),4))

Logistic :	 0.7947
D-Tree :	 0.7677
Forest :	 0.7936
N-Bayes :	 0.5217
LinearSVM :	 0.7845
rbf SVM :	 0.807
KNearestN :	 0.7846
XGBoost :	 0.8227
AdaBoost :	 0.8092


About the same